In [134]:
import pandas as pd
import numpy as np
import xgboost as xgb

import sklearn.preprocessing as skpp

import datetime

from mimicpreprocess import DataHandler

In [135]:
dh = DataHandler()
dh.connect()

In [136]:
admissions = dh.admissions_query()
patients = dh.patient_query()

In [159]:
patient_info.head()

,subject_id,hadm_id,admittime,admission_type,insurance,ethnicity,death_period,gender,dob,age
0,357,122609,2198-11-01 22:36:00,EMERGENCY,Private,WHITE,0,M,2135-03-22,63
1,366,134462,2164-11-18 20:27:00,EMERGENCY,Medicare,HISPANIC OR LATINO,0,M,2112-05-22,52
2,94,183686,2176-02-25 16:49:00,EMERGENCY,Medicare,ASIAN,0,M,2101-09-20,74
3,21,111970,2135-01-30 20:50:00,EMERGENCY,Medicare,WHITE,1,M,2047-04-04,87
4,353,108923,2151-03-28 16:01:00,EMERGENCY,Medicare,WHITE,0,M,2089-07-23,61


In [186]:
patient_ids = patient_info.subject_id.tolist()
patient_hadm_ids = patient_info.hadm_id.tolist()

hadm_id_tuple_list = zip(patient_ids, patient_hadm_ids)

prior_visits_query = dh.session.query(dh.Admission).filter(dh.Admission.subject_id.in_(patient_ids))#.filter(~dh.Admission.hadm_ids.in_(patient_hadm_ids))
prior_visits = pd.read_sql(prior_visits_query.statement, prior_visits_query.session.bind).groupby('subject_id')

admissions['prior'] = np.nan

for patient_id, group in prior_visits:
    patient_admissions = [item for item in hadm_id_tuple_list if item[0] == patient_id]

    for item in patient_admissions:
        admit_time = group.loc[group['hadm_id'] == item[1]].iloc[0]['admittime']
        priors = len(group[(group['admittime'] < admit_time)])
        admissions.loc[admissions.hadm_id == item[1], 'prior'] = priors

In [137]:
# Join patients and admission dataframes to obtain gender and age of patient
patient_info = admissions.join(patients.set_index('subject_id'), on='subject_id')

In [138]:
patient_info['age'] = patient_info.apply (lambda row: dh.age (row),axis=1)

In [139]:
patient_vitals = dh.lab_event_query(patient_info)

In [140]:
patient_vitals.isnull().sum()

oxygen                   836
pco2                     412
PH                       355
po2                      412
tempurature              634
lipase                   436
hematocrit                 9
hemoglobin                10
INR                       68
lymphocytes               58
alkaline phosphatase     198
MCH                       11
amylase                  627
neutrophils               58
BUN                        8
platelet                   8
bicarbonate               14
CRP                      993
PTT                       68
PT                        72
RBCDW                     10
calcium                   23
ESR                     1120
creatinine                 8
WBC                     1172
glucose                    9
AST                      183
lactate                   86
dtype: int64

Start Running here

In [141]:
from itertools import chain
# Vitals that will be kept. The rest dropped due to lack of data
keep_vitals = ['bicarbonate' ,'INR' ,'MCH' ,'AST','alkaline phosphatase' , 'creatinine', 'platelet', 'PT', 'PTT', 'lymphocytes', 'RBCDW', 'calcium', 'neutrophils', 'glucose', 'hematocrit', 'hemoglobin', 'lactate', 'BUN']
# keep_vitals = keep_vitals + list(chain.from_iterable((x + '_max', x + '_min') for x in keep_vitals))
patient_vitals_fixed = patient_vitals[keep_vitals]

In [142]:
 patient_vitals_fixed[patient_vitals_fixed.isnull().sum(axis=1) >= 6]

,bicarbonate,INR,MCH,AST,alkaline phosphatase,creatinine,platelet,PT,PTT,lymphocytes,RBCDW,calcium,neutrophils,glucose,hematocrit,hemoglobin,lactate,BUN
hadm_id,,,,,,,,,,,,,,,,,,
169179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176442,30.000000,NaN,34.100000,NaN,NaN,0.400000,143.000000,NaN,NaN,11.0,14.50,9.10,78.0,106.000000,36.000,12.80,NaN,17.0
194197,30.000000,NaN,29.100000,NaN,NaN,0.700000,147.000000,NaN,NaN,NaN,16.70,8.30,NaN,162.000000,31.200,11.00,NaN,19.0
129316,NaN,NaN,NaN,26.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.10,NaN
187308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134921,23.000000,NaN,29.450000,NaN,NaN,1.500000,260.500000,NaN,NaN,17.1,14.40,8.35,78.3,123.750000,39.150,12.90,NaN,81.5
152414,23.000000,NaN,29.433333,NaN,NaN,0.750000,363.333333,NaN,NaN,9.9,13.60,NaN,85.8,105.000000,25.975,8.60,0.95,16.0
169398,24.666667,NaN,26.900000,NaN,NaN,1.266667,377.000000,NaN,NaN,13.0,13.05,8.45,82.8,113.333333,31.350,10.75,NaN,23.0
197907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
# Find all hadm_ids where there are 6 or greater missing values. 
drop_hadm_ids = patient_vitals_fixed[patient_vitals_fixed.isnull().sum(axis=1) >= 6].index

In [144]:
patient_vitals_final = patient_vitals_fixed[~patient_vitals_fixed.index.isin(drop_hadm_ids)]

In [145]:
# Replace NaN values with the mean of each column. In the actual ML phase it may be worth figuring out WHY
# Some values are missing. Here we will simply continue with mean.
patient_vitals_final = patient_vitals_final.fillna(patient_vitals.mean())

In [146]:
# Add age column prior to standardization
keep_vitals.append('age')

In [147]:
patient_info = patient_info[~patient_info.hadm_id.isin(drop_hadm_ids)]

In [148]:
# New patient info dataframe finally with vitals
vital_patient_info = patient_info.join(patient_vitals_final, on='hadm_id')
# vital_patient_info.isnull().sum()
# Standardize the data from vital columns
stdsc = skpp.StandardScaler()
vital_patient_info[keep_vitals] = stdsc.fit_transform(vital_patient_info[keep_vitals])

In [149]:
potential_outliers = []
from collections import Counter
from IPython.display import display

# For each feature find the data points with extreme high or low values
for feature in patient_vitals_final.keys():

    # Calculate Q1
    Q1 = np.percentile(patient_vitals_final[feature], 25)
    
    # Calculate Q3
    Q3 = np.percentile(patient_vitals_final[feature], 75)
    
    step = 1.5*(Q3-Q1)
    
    # Display the outliers
    outlier = patient_vitals_final[~((patient_vitals_final[feature] >= Q1 - step) & (patient_vitals_final[feature] <= Q3 + step))]
    potential_outliers += outlier.index.tolist()
    
# print Counter(potential_outliers)
# len({k:v for (k,v) in Counter(potential_outliers).items() if v > 4})
outliers = [k for (k,v) in Counter(potential_outliers).items() if v > 4]

In [150]:
vital_patient_info = vital_patient_info[~vital_patient_info.index.isin(outliers)]

In [151]:
y_vitals = vital_patient_info['death_period']
X_vitals = vital_patient_info.drop(['death_period', 'subject_id', 'hadm_id', 'admittime', 'ethnicity', 'dob'], 1)
X_vitals = pd.get_dummies(X_vitals)

In [152]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import RandomizedSearchCV

param_grid = {
              "n_estimators": list(range(1,1000)),
             }

forest = RandomForestClassifier( random_state=42)

In [153]:
rand_for = RandomizedSearchCV(forest, param_grid, scoring = 'accuracy', n_iter=20, random_state=42)
_ = rand_for.fit(X_vitals,y_vitals)

In [154]:
print(rand_for.best_score_)
print(rand_for.best_params_)
print(rand_for.best_estimator_)

0.821917808219
{'n_estimators': 122}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=122, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)


In [155]:
importances = rand_for.best_estimator_.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_vitals.shape[1]):
    print "{}) {} {}".format(f, X_vitals.columns[indices[f]], importances[indices[f]])

0) lactate 0.112848063183
1) bicarbonate 0.0714871081807
2) BUN 0.0699732552175
3) PT 0.0566347543359
4) calcium 0.0552498666893
5) RBCDW 0.052879445113
6) creatinine 0.0479052531285
7) AST 0.0467770705944
8) platelet 0.0466520676304
9) hematocrit 0.0458771595783
10) age 0.0446118351371
11) alkaline phosphatase 0.0432309251201
12) PTT 0.0431457470703
13) MCH 0.0415636242685
14) neutrophils 0.0412849764704
15) hemoglobin 0.040515893532
16) glucose 0.0394940451161
17) lymphocytes 0.0393700587534
18) INR 0.0368596141203
19) gender_F 0.00553329104242
20) insurance_Private 0.00484775322714
21) insurance_Medicare 0.00476966047603
22) gender_M 0.00458036830205
23) insurance_Medicaid 0.00196060920857
24) admission_type_EMERGENCY 0.000596496492116
25) insurance_Government 0.000581188045311
26) insurance_Self Pay 0.000466617674557
27) admission_type_URGENT 0.000303252292218


## Extreme Gradient Boosting

In [156]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score

In [157]:
model = xgb.XGBClassifier(objective='binary:logistic')
# kfold = KFold(n_folds=10, random_state=42)
kfold = StratifiedKFold(y_vitals, n_folds=10, random_state=42)
results = cross_val_score(model, X_vitals, y_vitals, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 82.02% (2.51%)


In [124]:
# from scipy.stats import randint as sp_randint
from scipy.stats import expon

param_grid = {
                  "max_depth": list(range(1,20)),
                
             }

0.010000000000000002

In [ ]:
print(rand_XGB.best_score_)
print(rand_XGB.best_params_)
print(rand_XGB.best_estimator_)